In [27]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy.stats import chi2
from impyute.imputation.cs import mice
import statsmodels.api as sm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn import linear_model
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors, Lipinski, Draw
#from mordred import Calculator, descriptors
#import joblib
from sklearn.model_selection import KFold
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
data=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AML Group/missing_data.csv")
data.head(5)

,molecule_chembl_id,canonical_smiles,standard_type,standard_value,standard_units,MW,HeavyAtomMolWt,LogP,MolMR,NumHDonors,NumHAcceptors,NumAtoms,AR,TPSA,RB
0,CHEMBL133551,COCC1(c2ccc(Cl)cc2)c2ccccc2C2=NCCCN21,Inhibition,5.0,%,326.827,307.675,3.6959,93.2770,0.0,3.0,23.0,2.0,24.83,3.0
1,CHEMBL337609,COc1cc(C2(O)c3ccccc3C3=NCCN32)cc(OC)c1OC,Inhibition,5.7,%,340.379,320.219,1.9816,93.6488,1.0,6.0,25.0,2.0,63.52,4.0
2,CHEMBL336827,OC1(c2ccc(Cl)cc2)c2ccccc2-c2nc3ccccc3n21,Inhibition,30.0,%,332.790,319.686,4.4118,95.0178,1.0,3.0,24.0,4.0,38.05,1.0
3,CHEMBL441477,OC1(c2ccc(Cl)cc2)c2ccccc2C2=NCc3ccccc3CN21,Inhibition,12.0,%,360.844,343.708,4.3094,103.1188,1.0,3.0,26.0,3.0,35.83,1.0
4,CHEMBL131792,OC1(c2ccc(Cl)cc2)c2ccccc2C2=Nc3ccccc3CN21,Inhibition,45.0,%,346.817,331.697,4.4409,99.0748,1.0,3.0,25.0,3.0,35.83,1.0


In [11]:
data.shape

(1258, 15)

In [12]:
data_impute=data[["standard_value","MW","HeavyAtomMolWt","LogP","MolMR","NumHDonors","NumHAcceptors","NumAtoms","AR","TPSA","RB"]]
data_impute.shape

(1258, 11)

In [13]:
data_impute.isnull().sum()

standard_value    175
MW                  0
HeavyAtomMolWt      0
LogP                0
MolMR               0
NumHDonors          0
NumHAcceptors       0
NumAtoms            0
AR                  0
TPSA                0
RB                  0
dtype: int64

In [14]:
#MICE ALGORITHM WITH K-FOLD CROSS VALIDATION
#Finding training and validation loss using MICE
mice_imputer=IterativeImputer()
n_folds = 10
#cross validation object
kf = KFold(n_splits=n_folds, shuffle=True)
#training error list
train_errors = np.zeros(n_folds)
#validation error list
val_errors = np.zeros(n_folds)

for i, (train_index, val_index) in enumerate(kf.split(data_impute)):
    # split data into training and validation sets
    train_df = data_impute.iloc[train_index]
    val_df = data_impute.iloc[val_index]

    # impute missing values using MICE on training set
    train_imputed_array = mice_imputer.fit_transform(train_df)
    train_imputed_df = pd.DataFrame(train_imputed_array, columns=train_df.columns)

    # calculate training error
    train_error = ((train_df - train_imputed_df) ** 2).mean().mean()
    train_errors[i] = train_error

    # impute missing values using MICE on validation set
    val_imputed_array = mice_imputer.transform(val_df)
    val_imputed_df = pd.DataFrame(val_imputed_array, columns=val_df.columns)

    # calculate validation error
    val_error = ((val_df - val_imputed_df) ** 2).mean().mean()
    val_errors[i] = val_error


print("Average training error:", train_errors.mean())
print("Average validation error:", val_errors.mean())

Average training error: 3527.3132496545395
Average validation error: 5249.701339406072


In [15]:
#MICE ALGORITHM - imputing
mice_imputer=IterativeImputer()
imputed_array = mice_imputer.fit_transform(data_impute)
imputed_df = pd.DataFrame(imputed_array, columns=data_impute.columns)

In [16]:
print("missing in inh",imputed_df.standard_value.isna().sum())

missing in inh 0


In [17]:
#Midas
from sklearn.model_selection import train_test_split
train_df = data_impute.sample(frac=0.8, random_state=1)
val_df = data_impute.drop(train_df.index)

In [18]:
def midas_impute(df, k):
    """
    Implements the MIDAS algorithm for imputing missing values in a dataframe
    
    Parameters:
        df (pandas dataframe): The input dataframe to impute missing values in
        k (int): The number of previous time steps to use for imputation
        
    Returns:
        imputed_df (pandas dataframe): The imputed dataframe
    """
    
    # Create a copy of the input dataframe to store the imputed values
    imputed_df = df.copy()
    
    # Loop through each column in the dataframe
    for col in df.columns:
        
        # Check if the column has any missing values
        if df[col].isnull().sum() > 0:
            
            # Loop through each row in the dataframe
            for i in range(df.shape[0]):
                
                # Check if the value in the current row is missing
                if pd.isnull(df.loc[i, col]):
                    
                    # Calculate the start and end indices for the previous time steps
                    start_idx = max(0, i - k)
                    end_idx = i
                    
                    # Calculate the weighted average of the previous time steps
                    weights = np.arange(1, end_idx - start_idx + 2)
                    imputed_df.loc[i, col] = np.sum(df.loc[start_idx:end_idx, col] * weights) / np.sum(weights)
    
    return imputed_df


In [ ]:
# Impute missing values in the training set
train_imputed_df = midas_impute(train_df, k=3)

# Impute missing values in the validation set
val_imputed_df = midas_impute(val_df, k=3)


In [20]:
# Calculate the mean squared error for the training set
train_mse = np.mean((train_imputed_df - train_df)**2)
print("Training MSE:", train_mse)

# Calculate the mean squared error for the validation set
val_mse = np.mean((val_imputed_df - val_df)**2)
print("Validation MSE:", val_mse)


Training MSE: standard_value     1997.746102
MW                14972.462382
HeavyAtomMolWt    14070.032164
LogP                  3.715834
MolMR               882.639516
NumHDonors            2.343202
NumHAcceptors         7.612939
NumAtoms             66.729167
AR                    1.201754
TPSA               2891.682200
RB                    9.702851
dtype: float64
Validation MSE: standard_value     3153.357659
MW                18593.179650
HeavyAtomMolWt    18265.661357
LogP                  2.825693
MolMR               788.719822
NumHDonors            3.000000
NumHAcceptors        11.058824
NumAtoms             68.235294
AR                    1.352941
TPSA               3588.522459
RB                    6.882353
dtype: float64


/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [21]:
#Softimpute
train_df, val_df = train_test_split(data_impute, test_size=0.2, random_state=42)


In [22]:
from fancyimpute import SoftImpute

# assume 'train_df' and 'val_df' are the split dataframes
train_df_imputed = SoftImpute().fit_transform(train_df)
val_df_imputed = SoftImpute().fit_transform(val_df)


[SoftImpute] Max Singular Value of X_init = 17619.333117
[SoftImpute] Iter 1: observed MAE=4.025245 rank=3
[SoftImpute] Iter 2: observed MAE=4.041673 rank=3
[SoftImpute] Iter 3: observed MAE=4.051817 rank=3
[SoftImpute] Iter 4: observed MAE=4.057797 rank=3
[SoftImpute] Iter 5: observed MAE=4.061322 rank=3
[SoftImpute] Iter 6: observed MAE=4.063550 rank=3
[SoftImpute] Iter 7: observed MAE=4.064941 rank=3
[SoftImpute] Iter 8: observed MAE=4.065811 rank=3
[SoftImpute] Iter 9: observed MAE=4.066372 rank=3
[SoftImpute] Iter 10: observed MAE=4.066742 rank=3
[SoftImpute] Iter 11: observed MAE=4.066990 rank=3
[SoftImpute] Iter 12: observed MAE=4.067159 rank=3
[SoftImpute] Iter 13: observed MAE=4.067274 rank=3
[SoftImpute] Iter 14: observed MAE=4.067355 rank=3
[SoftImpute] Iter 15: observed MAE=4.067411 rank=3
[SoftImpute] Iter 16: observed MAE=4.067450 rank=3
[SoftImpute] Iter 17: observed MAE=4.067478 rank=3
[SoftImpute] Iter 18: observed MAE=4.067498 rank=3
[SoftImpute] Iter 19: observed MAE

In [25]:
#DAE
from sklearn.model_selection import train_test_split

# Split data into train and validation sets (80/20 split)
train_data, val_data = train_test_split(data_impute, test_size=0.2, random_state=42)


In [28]:
#DAE
# Scale the data
scaler = MinMaxScaler()
train_data_scaled = scaler.fit_transform(train_data)
val_data_scaled = scaler.transform(val_data)

# Create the DAE model
input_layer = Input(shape=(train_data.shape[1],))
encoded = Dense(32, activation='relu')(input_layer)
decoded = Dense(train_data.shape[1], activation='sigmoid')(encoded)
dae = Model(input_layer, decoded)

# Compile the DAE model
dae.compile(optimizer='adam', loss='binary_crossentropy')

# Train the DAE model
dae.fit(train_data_scaled, train_data_scaled, epochs=100, batch_size=32, validation_data=(val_data_scaled, val_data_scaled))


Epoch 1/100
32/32 [==============================] - 1s 9ms/step - loss: nan - val_loss: nan
Epoch 2/100
32/32 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 3/100
32/32 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 4/100
32/32 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 5/100
32/32 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 6/100
32/32 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 7/100
32/32 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 8/100
32/32 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 9/100
32/32 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 10/100
32/32 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 11/100
32/32 [==============================] - 0s 3ms/step - l

In [29]:
# Impute missing values in the training and validation sets
train_data_imputed = dae.predict(train_data_scaled)
val_data_imputed = dae.predict(val_data_scaled)


8/8 [==============================] - 0s 4ms/step


In [30]:
# Compute the training and validation errors
train_error = np.mean(np.abs(train_data_scaled - train_data_imputed))
val_error = np.mean(np.abs(val_data_scaled - val_data_imputed))

print(f'Training error: {train_error:.4f}')
print(f'Validation error: {val_error:.4f}')


Training error: nan
Validation error: nan


In [31]:
#AE
# Step 1: Split data into train and validation sets
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(data_impute, test_size=0.2, random_state=42)

# Step 2: Impute missing values using AE
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Define the AE model
input_dim = train_data.shape[1]
encoding_dim = 64
input_layer = Input(shape=(input_dim,))
encoder_layer = Dense(encoding_dim, activation='relu')(input_layer)
decoder_layer = Dense(input_dim, activation='sigmoid')(encoder_layer)
autoencoder = Model(inputs=input_layer, outputs=decoder_layer)

# Compile and fit the AE model only on non-missing data from train set
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(train_data[train_data.notnull()], train_data[train_data.notnull()], epochs=10, batch_size=32)

# Step 3: Compute imputed values for missing data in train and validation sets
train_data_imputed = autoencoder.predict(train_data)
val_data_imputed = autoencoder.predict(val_data)

# Step 4: Compute reconstruction error for train and validation sets
import numpy as np

# Compute MSE for train set
mse_train = np.mean(np.power(train_data - train_data_imputed, 2), axis=1)

# Compute MSE for validation set
mse_val = np.mean(np.power(val_data - val_data_imputed, 2), axis=1)

# Step 5: Print MSE for train and validation sets
print('Training error:', np.mean(mse_train))
print('Validation error:', np.mean(mse_val))


Epoch 1/10
32/32 [==============================] - 2s 7ms/step - loss: nan
Epoch 2/10
32/32 [==============================] - 0s 6ms/step - loss: nan
Epoch 3/10
32/32 [==============================] - 0s 7ms/step - loss: nan
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: nan
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: nan
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: nan
Epoch 7/10
32/32 [==============================] - 0s 4ms/step - loss: nan
Epoch 8/10
32/32 [==============================] - 0s 3ms/step - loss: nan
Epoch 9/10
32/32 [==============================] - 0s 3ms/step - loss: nan
Epoch 10/10
8/8 [==============================] - 0s 4ms/step
Training error: nan
Validation error: nan
